<a href="https://colab.research.google.com/github/kabirwalia8300/Arg_gen/blob/master/huggingfaceTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine Tuning Transformer XL using Arguana Corpus for Argument Generation

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
# !git clone https://github.com/huggingface/transformers.git
!ls

gdrive	sample_data  test.csv  train.csv  transformers	val.csv


In [ ]:
# !pip install ./transformers/

In [3]:
#imports
import pandas as pd
import numpy as np
import torch
import os, sys
from sklearn.model_selection import train_test_split

#Torchtext
from torchtext.data import Field, BucketIterator, TabularDataset

#Huggingface
from transformers import TransfoXLConfig, TransfoXLModel, TransfoXLTokenizer, TransfoXLLMHeadModel
from transformers.tokenization_transfo_xl import LMOrderedIterator
from tqdm import tqdm

In [ ]:
url = 'https://raw.githubusercontent.com/kabirwalia8300/Arg_gen/master/Seq2Seq/args2.csv'
df = pd.read_csv(url, error_bad_lines=False).iloc[:, 1:]
# removing duplicates
df = df.drop_duplicates(subset='Arguments', keep="first")
df['Arguments'] = [x.split('\n\n', 1)[1] for x in df['Arguments']]
df_train, test = train_test_split(df, test_size=0.02)
train, val = train_test_split(df_train, test_size = 0.05)
train = train[['Arguments', 'Counters']]
val = val[['Arguments', 'Counters']]
test = test[['Arguments', 'Counters']]
train.to_csv('./train.csv', header=False, index=False)
val.to_csv('./val.csv', header=False, index=False)
test.to_csv('./test.csv', header=False, index=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
# Transformer XL Tokenizer
tok = TransfoXLTokenizer.from_pretrained("transfo-xl-wt103", do_lower_case=True)

In [ ]:
# ARGUMENT = Field(tokenize=tok.encode,
#                 use_vocab=False,
#                 init_token=tok.cls_token_id,
#                 eos_token=tok.sep_token_id,
#                 include_lengths=True,
#                 pad_token=tok.pad_token_id,
#                 unk_token=tok.unk_token_id,
#                 batch_first=True,
#                 )
# COUNTER = Field(tokenize=tok.encode,
#                 use_vocab=False,
#                 init_token=tok.cls_token_id,
#                 eos_token=tok.sep_token_id,
#                 pad_token=tok.pad_token_id,
#                 unk_token=tok.unk_token_id,
#                 batch_first=True,
#                 )

In [ ]:
# #set up vocab for article and summary from train data.
# ARGUMENT.build_vocab(train_data, min_freq=2)
# COUNTER.build_vocab(train_data, min_freq=2)

In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = torch.load('data/' + ID + '.pt')
        y = self.labels[ID]

        return X, y

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)